In [ ]:
import os
import ast
import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor
import numpy as np
import pandas as pd
import soundfile as sf
from src.text_processing import *
from src.data_processing import *
from src.ui_tools import *
from src.audio_processing import *

# Add espeak's shared library directory
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib'

In [2]:
readingTestFluencE_df = prepare_readingTest_data(test_type='FluencE')

/Users/cleo/EPFL/Ma2/Project/Rhythmico-ASR/src/data_processing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTest_df['testResults'] = readingTest_df['testResults'].apply(lambda x: convert_str_to_dct_eval(x))
/Users/cleo/EPFL/Ma2/Project/Rhythmico-ASR/src/data_processing.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTest_df['evaluationResults'] = readingTest_df['evaluationResults'].apply(lambda x: convert_str_to_dct_eval(x))


In [ ]:
# We extract the audio files and convert them to .wav format
save_recordings_as_wav(readingTestFluencE_df, 
                       output_dir='wav_files/readingTestFluencE',
                       target_sample_rate=16000, 
                       channels=1)

In [ ]:
# If the transcription file already exists, we do nothing
transcription_file = 'testPhoneme_deletion_transcriptions.csv'
file_exists = os.path.isfile(transcription_file)

if not(file_exists):
    # We extract the text that is read by the children during the readingTestFluencE tests
    test = readingTestFluencE_df[readingTestFluencE_df.iloc[0]]['testParameters'].values[0]
    test_dict = ast.literal_eval(test)
    selected_text = test_dict['textSelected']['text']

    save_phonetic_transcription_to_csv(selected_text, test_type='readingTestFluencE', folder='transcriptions', file_name='readingTestFluencE_transcriptions.csv')

Phonetic transcription saved to transcriptions/readingTestFluencE_transcriptions.csv


In [3]:
# Extract recordings and their corresponding evaluation results (e.g., 'wordsState')
recordings = readingTestFluencE_df['testResults'].apply(
    lambda x: x['recording'] if 'recording' in x else None).dropna().tolist()

evaluation_results = readingTestFluencE_df['evaluationResults'].apply(
    lambda x: x['wordsState'] if 'wordsState' in x else None).dropna().tolist()

# Create the interactive audio player with evaluation results
create_audio_player_with_results(recordings, evaluation_results)

Output()

Output()